In [117]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import warnings
from loguru import logger
import multiprocessing as mp
import os, inspect, sys
import redis
from mci_model.config.base import config, DATASET_DIR

In [118]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [119]:
r = redis.StrictRedis(os.getenv('REDIS_HOST'), os.getenv('REDIS_PORT'), decode_responses=True)

In [120]:
window_size = 49131
last_rec = int(r.get('last_id'))
live_data = pd.DataFrame([r.hgetall(rec) for rec in range(1, window_size)])

In [121]:
train_data = pd.read_csv(DATASET_DIR / f'{config.app_config.test_data_file}',
                         usecols=live_data.columns).reset_index().drop(columns=['index'])

In [122]:
def setup_metrics_table(train_data):
    metrics_index = ['Median', 
                      'Mean', 
                      'Stdev', 
                      "New Cat Detected", 
                      "Null Val Detected", 
                      "Fisher p-val (Missing/Non-Missing)", 
                      "Extreme Val Detected",
                      'Chi Square p-val (Cat. Col. Dist.)',
                      'KS 2-Sample p-val (Cont. Col. Dist.)',
                      'Drift Detected'
                     ]

    columns = list(train_data.columns[1:])


    arr = np.zeros(shape=(len(metrics_index), len(columns)))
    metrics_table = pd.DataFrame(arr, index = metrics_index, columns = columns)
    return metrics_table


In [123]:
def calc_metrics(metrics_table, train_data, live_data, col):
#     metrics_table, train_data, live_data, col = args
    cat_columns = ['premises_type', 'occurrencemonth', 'occurrencehour',
       'occurrencedayofweek', 'MCI', 'Neighbourhood', 'occurrenceday', 'Pub_Id', 'Park_Id', 'PS_Id', 'occurrencedayofyear']
    dtypes={'occurrencedate':'object',
    'occurrencehour':'int',
    'premises_type':'object',
    'occurrencemonth':'object',
    'occurrencedayofweek':'object',
    'MCI':'object',
    'Neighbourhood':'object', 
    'occurrenceday':'int',
    'occurrencedayofyear':'int',
    'Pub_Id':'int',
    'Park_Id':'int',
    'PS_Id':'int',
    'Lat':'float',
    'Long':'float'}
        

    num_columns = ['Lat', 'Long']
    target_columns = ['Lat', 'Long']
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
#         for col in metrics_table.columns:
        try:
            live_data[col] = live_data[col].astype(train_data[col].dtype)
        except:
            live_data[col] = live_data[col].apply(lambda x: float(x))
            live_data[col].astype(train_data[col].dtype)
            
        print(f"Processing {col}")
            # median, mean, std. dev.

        if col in cat_columns:
            metrics_table[col]['Median'] = live_data[col].value_counts().index[0]
            metrics_table[col]['Mean'] = 'N/A'
            metrics_table[col]['Stdev'] = 'N/A'
        else: 
            metrics_table[col]['Mean'] = live_data[col].mean()
            metrics_table[col]['Median'] = 'N/A'
            metrics_table[col]['Stdev'] = live_data[col].std()




        # New categories

        uniques = list(live_data[col].unique())
        if len([x for x in live_data[col] if x not in uniques]) > 0:
            metrics_table[col]['New Cat Detected'] = True
        else:
            metrics_table[col]['New Cat Detected'] = False



        # Missing values dist: Fisher
        train_data[f'{col}_na'] = np.where(train_data[col].isnull(), 1, 0)
        live_data[f'{col}_na'] = np.where(live_data[col].isnull(), 1, 0)
        ct = pd.concat([train_data.groupby(f'{col}_na')[f'{col}_na'].count(),
                        live_data.groupby(f'{col}_na')[f'{col}_na'].count()], axis=1)
        ct.fillna(0, inplace=True)

        if live_data[f'{col}_na'].sum() > 0 or train_data[f'{col}_na'].sum() > 0:
            metrics_table[col]['Null Val Detected'] = True
            _, metrics_table[col]['Fisher p-val (Missing/Non-Missing)'] = stats.fisher_exact(ct)
        else:
            metrics_table[col]['Null Val Detected'] = False
            metrics_table[col]['Fisher p-val (Missing/Non-Missing)'] = False



        # Extreme values cont. col.:

        if col in num_columns:   
            train_extremes = train_data[col].agg(['min', 'max'])
            live_extremes = live_data[col].agg(['min', 'max'])
            if train_extremes[0] > live_extremes[0] or train_extremes[1] < live_extremes[1]:
                metrics_table[col]['Extreme Val Detected'] = True
            else:
                metrics_table[col]['Extreme Val Detected'] = False
        else: 
            metrics_table[col]['Extreme Val Detected'] = 'N/A'



        # Cat feature distributions: Chi Square
        if col in cat_columns:
            ct = train_data[col].fillna('Missing').value_counts()
            cl = live_data[col].fillna('Missing').value_counts()
            if 'Missing' not in ct.index:
                ct.at['Missing'] = 0.1
            if 'Missing' not in cl:
                cl.at['Missing'] = 0.1
            for element in ct.index:
                if element not in cl.index:
                    cl.at[element] = 0.1

            for element in cl.index:
                if element not in ct.index:
                    ct.at[element] = 0.1
            ct = ct.apply(lambda x: (x/sum(ct))*sum(cl)) # scale to the size of cl to use a chisquare test
            p_val = stats.chisquare(f_obs=ct, f_exp=cl)[1]
            metrics_table[col]['Chi Square p-val (Cat. Col. Dist.)'] = round(p_val, 4)
        else:
            metrics_table[col]['Chi Square p-val (Cat. Col. Dist.)'] = 'N/A'

        # Cont. vals (Long, Lat):  2 sample K-S distance p-val.
        if col in num_columns:
            _, p_val = stats.ks_2samp(train_data[col], live_data[col])
            metrics_table[col]['KS 2-Sample p-val (Cont. Col. Dist.)'] = round(p_val, 4)
        else: 
            metrics_table[col]['KS 2-Sample p-val (Cont. Col. Dist.)'] = 'N/A'

        # Drift detected
        if col in cat_columns:
            metrics_table[col]['Drift Detected'] = (metrics_table[col]['Chi Square p-val (Cat. Col. Dist.)'] < 0.05)
        elif col in num_columns:
            metrics_table[col]['Drift Detected'] = (metrics_table[col]['KS 2-Sample p-val (Cont. Col. Dist.)'] < 0.05)

    return metrics_table

In [124]:
live_data.columns

Index(['occurrencedate', 'premises_type', 'occurrencehour', 'occurrenceday',
       'occurrencedayofyear', 'occurrencemonth', 'occurrencedayofweek', 'MCI',
       'Neighbourhood', 'Pub_Id', 'Park_Id', 'PS_Id', 'Lat', 'Long'],
      dtype='object')

In [125]:
metrics_table = setup_metrics_table(train_data)
args = [(metrics_table[[col]], train_data[[col]], live_data[[col]], col) for col in live_data.columns[1:]]
with mp.Pool(mp.cpu_count())as pool:
    metrics_table = pd.concat(pool.starmap(calc_metrics, args), 1)

metrics_table.reset_index(level=0, inplace=True)
metrics_table.rename(columns={'index': 'Metric Name'}, inplace=True)

Processing premises_type
Processing occurrencehour
Processing occurrenceday
Processing occurrencemonth
Processing occurrencedayofyearProcessing occurrencedayofweek

Processing MCI
Processing NeighbourhoodProcessing Pub_Id

Processing Park_IdProcessing PS_Id

Processing Long
Processing Lat


/tmp/ipykernel_15380/380468474.py:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  metrics_table = pd.concat(pool.starmap(calc_metrics, args), 1)


In [116]:
metrics_table

,Metric Name,premises_type,occurrencehour,occurrenceday,occurrencedayofyear,occurrencemonth,occurrencedayofweek,MCI,Neighbourhood,Pub_Id,Park_Id,PS_Id,Lat,Long
0,Median,Commercial,12.0,24.0,83.0,March,Friday,Assault,West Humber-Clairville,0.0,0.0,0.0,N/A,N/A
1,Mean,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,43.72701,-79.421583
2,Stdev,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.000802,0.001184
3,New Cat Detected,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Null Val Detected,False,False,False,False,True,True,False,False,False,False,False,False,False
5,Fisher p-val (Missing/Non-Missing),False,False,False,False,0.001125,0.001125,False,False,False,False,False,False,False
6,Extreme Val Detected,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,False
7,Chi Square p-val (Cat. Col. Dist.),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N/A,N/A
8,KS 2-Sample p-val (Cont. Col. Dist.),N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.0,0.0
9,Drift Detected,True,True,True,True,True,True,True,True,True,True,True,True,True


In [ ]:
# 
metrics_table = calc_metrics(metrics_table, train_data[['Lat']], live_data[['Lat']], 'Lat')
metrics_table.reset_index(level=0, inplace=True)
metrics_table.rename(columns={'index': 'Metric Name'}, inplace=True)

In [32]:
metrics_table

,Metric Name,occurrencehour,premises_type,occurrencemonth,occurrencedayofweek,MCI,Neighbourhood,occurrenceday,occurrencedayofyear,Pub_Id,Park_Id,PS_Id,Lat,Long
0,Median,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N/A,0.0
1,Mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.72686,0.0
2,Stdev,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000951,0.0
3,New Cat Detected,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
4,Null Val Detected,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
5,Fisher p-val (Missing/Non-Missing),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
6,Extreme Val Detected,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7,Chi Square p-val (Cat. Col. Dist.),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N/A,0.0
8,KS 2-Sample p-val (Cont. Col. Dist.),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0006,0.0
9,Drift Detected,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
